In [ ]:
"""
Autonomous Delivery Agent implementation
"""

import time
from typing import List, Tuple, Dict, Callable
from .algorithms.uninformed_search import breadth_first_search, uniform_cost_search
from .algorithms.informed_search import a_star_search, adaptive_a_star
from .algorithms.local_search import hill_climbing, simulated_annealing

class DeliveryAgent:
    def __init__(self, environment):
        self.env = environment
        self.position = environment.agent_start
        self.path = []
        self.fuel_used = 0
        self.time_elapsed = 0
        self.delivered_packages = 0
        self.algorithm_stats = {}

    def plan_path(self, algorithm: str, start: Tuple[int, int], goal: Tuple[int, int],
                 **kwargs) -> Dict[str, any]:
        """Plan path using specified algorithm"""

        start_time = time.time()

        if algorithm == 'bfs':
            result = breadth_first_search(self.env, start, goal, **kwargs)
        elif algorithm == 'ucs':
            result = uniform_cost_search(self.env, start, goal, **kwargs)
        elif algorithm == 'astar':
            heuristic = kwargs.get('heuristic', 'manhattan')
            if heuristic == 'manhattan':
                from .utils import manhattan_distance
                result = a_star_search(self.env, start, goal, manhattan_distance, **kwargs)
            else:
                from .utils import euclidean_distance
                result = a_star_search(self.env, start, goal, euclidean_distance, **kwargs)
        elif algorithm == 'hillclimb':
            result = hill_climbing(self.env, start, goal, **kwargs)
        elif algorithm == 'annealing':
            result = simulated_annealing(self.env, start, goal, **kwargs)
        else:
            raise ValueError(f"Unknown algorithm: {algorithm}")

        end_time = time.time()
        result['planning_time'] = end_time - start_time

        self.algorithm_stats[algorithm] = result
        return result

    def execute_path(self, path: List[Tuple[int, int]]) -> Dict[str, any]:
        """Execute a planned path"""
        if not path:
            return {'success': False, 'fuel_used': 0, 'time_elapsed': 0}

        self.path = path
        fuel_used = 0
        time_elapsed = 0

        for i in range(len(path)):
            self.position = path[i]

            # Check if position becomes invalid during execution (dynamic obstacle)
            if not self.env.is_valid_position(*self.position, time_elapsed):
                return {
                    'success': False,
                    'fuel_used': fuel_used,
                    'time_elapsed': time_elapsed,
                    'reason': 'Dynamic obstacle blocked path'
                }

            if i > 0:
                fuel_used += self.env.get_terrain_cost(*path[i])
            time_elapsed += 1

            # Check if we reached a delivery point
            if self.position in self.env.delivery_points:
                self.delivered_packages += 1

        self.fuel_used += fuel_used
        self.time_elapsed += time_elapsed

        return {
            'success': True,
            'fuel_used': fuel_used,
            'time_elapsed': time_elapsed,
            'packages_delivered': self.delivered_packages
        }

    def replan_for_dynamic_obstacle(self, current_position: Tuple[int, int],
                                   goal: Tuple[int, int], original_algorithm: str) -> Dict[str, any]:
        """Replan when dynamic obstacle is detected"""
        print("Dynamic obstacle detected! Replanning...")

        # Try original algorithm first
        result = self.plan_path(original_algorithm, current_position, goal)

        if not result['success']:
            # Fall back to local search
            result = self.plan_path('annealing', current_position, goal)

        return result

    def deliver_packages(self, algorithm: str = 'astar') -> Dict[str, any]:
        """Execute complete delivery mission"""
        results = {}
        total_fuel = 0
        total_time = 0

        for i, goal in enumerate(self.env.delivery_points):
            print(f"Delivering to goal {i+1} at {goal}")

            result = self.plan_path(algorithm, self.position, goal)
            execution_result = self.execute_path(result['path'])

            results[f'goal_{i+1}'] = {
                'planning': result,
                'execution': execution_result
            }

            total_fuel += execution_result['fuel_used']
            total_time += execution_result['time_elapsed']

        return {
            'results': results,
            'total_fuel': total_fuel,
            'total_time': total_time,
            'total_packages': self.delivered_packages
        }